## Naïve Bayes – Additional Advice
This notebook guides you through multiple steps you can follow to create a naïve Bayes classifier if you wish to implement this classifier for the assignment. After following these steps you will still need to collate and move your code into the main assignment notebook file so that it meets the required format.

Read each step (including the maths!) carefully.

You can implement a naïve Bayes classifier without following this advice.

**Note!** This notebook guides you only through the compulsory spam filtering part of the assignment. If you wish to do the optional part of the assignment on feature engineering for digit classification, you should be able to reuse this code for your classifier for this task. In this case, you should make sure the code is generalisable to more than 2 classes and arbitrary number of features.  

This notebook will not be graded and **does not need to be submitted**.

In [3]:
import numpy as np

training_spam = np.loadtxt(open("training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


## The model:  naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* $C = 1$ for spam messages
* $C = 0$ for ham messages


The classifier builds a model for the probability $p(C=c\ |\ \text{message})$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate
$\require{color}$
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \  \textcolor{blue}{p(C=c\ |\ \text{message})}.
\end{equation}
Using Bayes' rule we can write

\begin{equation}
p(C=c\ |\ \text{message}) = \frac{p(\text{message}\ |\ C=c)p(C=c)}{p(\text{message}\ |\ C=1)p(C=1) + p(\text{message}\ |\ C=0)p(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it as it does not influence which class would get the maximum $\textcolor{blue}{p(C=c\ |\ \text{message})}$. This gives us

\begin{equation}
\textcolor{blue}{p(C=c\ |\ \text{message})} \propto \textcolor{orange}{p(\text{message}\ |\ C=c)}\textcolor{green}{p(C=c)},
\end{equation}

where $\propto$ means "proportional to". I.e., the above is equivalent to:

\begin{equation}
\textcolor{blue}{p(C=c\ |\ \text{message})} = Const * \textcolor{orange}{p(\text{message}\ |\ C=c)}\textcolor{green}{p(C=c)},
\end{equation}

where $Const$ stands for some constant value.

The class priors $\textcolor{green}{p(C=c)}$ can be computed directly (you will do so in exercise A) but we need to further simplify $\textcolor{orange}{p(\text{message} \ |\ C=c)}$.


### Choice of the event model: *Multinomial* naïve Bayes

Different naïve Bayes models differ in their distributional assumptions about $\textcolor{orange}{p(\text{message}\ |\ C=c)}$. We represent a message using a **binary** [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as a set of $k$ keywords, that is, $message = (w_1, ..., w_k)$, where $w_i = 1$ if the  keyword $w_i$ appears in the message and $w_i = 0$ otherwise. In this particular assignment we have $k=54$ keywords.

We assume that the $p(w_1, ..., w_k |\ C=c)$ follows a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) for each class. Don't let the name scare you, this model simply assigns probabilities to different counts of events with multiple outcomes. So for example: "I roll a biased six-sided die six times, what is the probability that I get each side occurring exactly once" is a question that can be answered with a multinomial distribution. You don't need to understand all of the equations on the Wikipedia page.

Since this is a naïve Bayes model, we assume conditional indepence of keywords given the class, i.e., $p(w_1, ..., w_k |\ C=c) = \prod_{i=1}^k p(w_i |\ C=c)$. Intuitively, we assume that the words of a message were "drawn" independently from a bag of $k$ different words. Depending on the class membership $c$, each keyword $w$ has a probability $\theta_{c, w}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy,... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment,... $\}$, if the training data was mostly gathered from emails of researchers.

Under these assumptions, the probability of a whole message (it is also called *likelihood*), given that it belongs to class $c$, is then proportional to
\begin{equation}
\textcolor{orange}{p(\text{message}\ |\ C=c)} \propto \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}.
\end{equation}


The parameters $\textcolor{brown}{\theta_{c, w}}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$ (add-one smoothing), that is,
\begin{equation}
\textcolor{brown}{\theta_{c, w}} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $n_{c, w}$ is the number of times the keyword $w$ appears in messages of class $c$ in the training set and $n_{c}$ is the total count of keywords for all messages of class $c$, that is, $n_{c} = \sum_w n_{c, w}$.



We are now finally able to rewrite the *maximum a posteriori* estimate in a form that is easy to compute:
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}\right].
\end{equation}


#### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions. We can leverage this because the logarithm function does not change the position of the maximum of the input function, i.e.,

\begin{equation}
\underset{x}{\operatorname{argmax}}\log (f(x)) = \underset{x}{\operatorname{argmax}} f(x)
\end{equation}

for any function $f(x)$. 

Probability computation often involves taking a product of many terms (as in our case we need to take a product of $k$ terms of keyword probabilities and the prior probability of the class). Each term would be less than 1 as it represent some probability. When we multiply several terms less than 1, we can end up dealing with very small numbers close to 0, which is not a good idea when processing numbers on a computer. Using the logarithm in this case helps in two folds: (i) the logarithm of a product is a sum of the logarithms, i.e., we are dealing with summation of numbers rather than multiplication, which is much easier from the numerical stability point of view, (ii) the logarithm converts probabilities that are numbers from $[0, 1]$ into numbers from $(-\infty, 0]$, so that we do not process very small numbers close to 0.

This trick of using the logarithm of probabilities instead of probabilities themselves is often used in probabilistic modelling.

In our case, it results as

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \log \left( \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}}\right)^{w_i} \right) \\
 = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \log( \textcolor{green}{p(C=c)}) + \sum_{i = 1}^k w_i \ \log \left(\textcolor{brown}{\theta_{c, w_i}} \right) \right].
\end{equation}

## Part A: Estimate class priors

Note that for defining the empirical class priors we do not need input features, but only response variable. 

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) and returns a numpy array containing the **logarithm** of the empirical class priors $\textcolor{green}{p(C=c)}$ for $c \in \{0, 1\}$.

In [4]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s), 
    calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
        log(p(C=0)) and log(p(C=1))

    :param data: a numpy array of length n_samples
                 that contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    ### YOUR CODE HERE...
    
    # Defining np array
    out = np.array([])
    # Finding the probability of a ham email (c = 0)
    out = np.append(out, (len(data) - np.count_nonzero(data))/len(data))
    # Finding probability of a spam email (c = 1)
    out = np.append(out, np.count_nonzero(data)/len(data))
    # Getting the log of each element in out
    out = np.log(out)
    
    # Returning the np array
    return out

In [5]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam[:, 0])
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))

result [-0.48939034 -0.94933059]


## Part B: Estimate class-conditional likelihoods
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with input features and with binary response variable (0s and 1s) and returns the **logarithm** of the empirical class-conditional likelihoods $\log \left(\textcolor{brown}{\theta_{c, w_i}} \right)$ for all words $w_i$ and both classes ($c \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial event model and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [8]:
def estimate_log_class_conditional_likelihoods(input_data, labels, alpha=1.0):
    """
    Given input_data of binary features (words) and labels 
    (binary response variable (0s and 1s)), calculate the logarithm 
    of the empirical class-conditional likelihoods, that is,
    log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param input_data: a two-dimensional numpy-array with shape = [n_samples, n_features]
                       contains binary features (words)
    :param labels: a numpy array of length n_samples 
                   contains response variable

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    ### YOUR CODE HERE...
    
    # Getting the magnitude of the dimensions
    n_samples = input_data.shape[0]
    n_features = input_data.shape[1]
    
    # Isolating all the lines in input_data where the class label == 1
    # np.squeeze removes all dimensions of size 1
    class_ones = np.squeeze(input_data[np.where(labels == 1), :])
    
    # Isolating all the lines in input_data where the class label == 0
    class_zeros = np.squeeze(input_data[np.where(labels == 0), :])
    
    # Calculating the empircal conditional liklihoods for class = 0 (c == 0)
    class_zeros_conditionals = ((np.count_nonzero(class_zeros, axis=0)) + alpha) / ((n_samples - np.count_nonzero(labels)) + n_features*alpha)
    log_class_zeros_conditionals = np.log(class_zeros_conditionals)
    
    # Calculating the empircal conditional liklihoods for class = 1 (c == 1)
    class_ones_conditionals = (np.count_nonzero(class_ones, axis=0) + alpha) / (np.count_nonzero(labels) + n_features*alpha)
    log_class_ones_conditionals = np.log(class_ones_conditionals)
    
    theta = np.array([log_class_zeros_conditionals, log_class_ones_conditionals])
    
    return theta


In [9]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(
    training_spam[:, 1:], training_spam[:, 0], alpha=1.0)
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)

[[-1.82996121 -2.39191618 -1.36699161 -5.80964287 -1.56114762 -2.34390696
  -3.93784069 -2.78921798 -2.513806   -1.87781723 -2.86520389 -0.92684094
  -2.10834089 -3.17058554 -4.0178834  -2.39191618 -2.32840278 -2.10834089
  -0.66214839 -3.93784069 -1.15568252 -4.71103058 -3.6695767  -3.93784069
  -1.05605267 -1.35529557 -1.40292362 -1.93844185 -2.10834089 -1.99193054
  -2.37565566 -2.63158904 -2.28328234 -2.61096975 -1.94891315 -1.86806106
  -1.33230605 -4.0178834  -2.2542948  -2.39191618 -3.17058554 -2.17205671
  -2.32840278 -2.37565566 -1.30429301 -1.8488297  -4.0178834  -2.89187213
  -1.77540223 -0.67384443 -2.00298038 -1.3210065  -2.34390696 -2.53249813]
 [-1.09861229 -1.29325433 -0.6423075  -4.14313473 -0.55171061 -1.00764051
  -1.02644984 -1.14740245 -1.33545468 -0.87410912 -1.33545468 -0.59598343
  -1.26073114 -2.19722458 -2.08171169 -0.69088217 -0.95324644 -1.17638999
  -0.2368424  -1.61170806 -0.33647224 -3.1446059  -1.24485779 -1.03918887
  -3.60413823 -3.8918203  -5.39589769

## Part C: Combine intermediate steps into train method
Define a function called `train()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns both the **logarithm** of the empirical class priors $\textcolor{green}{p(C=c)}$ for $c \in \{0, 1\}$ and the **logarithm** of the empirical class-conditional likelihoods $\log \left(\textcolor{brown}{\theta_{c, w_i}} \right)$ for all words $w_i$ and both classes ($c \in {0, 1}$). 

Having implemented `estimate_log_class_priors()` and `estimate_log_class_conditional_likelihoods` you just need to call them.

In [50]:
def train(input_data, labels, alpha=1.0):
    """
    Given input_data of binary features (words) and labels 
    (binary response variable (0s and 1s)), calculate 
    * the logarithm of the empirical class priors, that is, 
      the logarithm of the proportions of 0s and 1s:
        log(p(C=0)) and log(p(C=1))
    * the logarithm of the empirical class-conditional likelihoods, 
      that is, log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]

    :return 
        log_class_priors: a numpy array of length two
    
        theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    ### YOUR CODE HERE...
    
    log_class_priors = estimate_log_class_priors(labels)
    theta = estimate_log_class_conditional_likelihoods(input_data, labels, alpha)
    
    return log_class_priors, theta

In [51]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors, log_class_conditional_likelihoods = train(
    training_spam[:, 1:], training_spam[:, 0], alpha=1.0)
print("log class priors")
print(log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))

print("log class conditional likelihoods")
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)

log class priors
[-0.48939034 -0.94933059]
log class conditional likelihoods
[[-1.82996121 -2.39191618 -1.36699161 -5.80964287 -1.56114762 -2.34390696
  -3.93784069 -2.78921798 -2.513806   -1.87781723 -2.86520389 -0.92684094
  -2.10834089 -3.17058554 -4.0178834  -2.39191618 -2.32840278 -2.10834089
  -0.66214839 -3.93784069 -1.15568252 -4.71103058 -3.6695767  -3.93784069
  -1.05605267 -1.35529557 -1.40292362 -1.93844185 -2.10834089 -1.99193054
  -2.37565566 -2.63158904 -2.28328234 -2.61096975 -1.94891315 -1.86806106
  -1.33230605 -4.0178834  -2.2542948  -2.39191618 -3.17058554 -2.17205671
  -2.32840278 -2.37565566 -1.30429301 -1.8488297  -4.0178834  -2.89187213
  -1.77540223 -0.67384443 -2.00298038 -1.3210065  -2.34390696 -2.53249813]
 [-1.09861229 -1.29325433 -0.6423075  -4.14313473 -0.55171061 -1.00764051
  -1.02644984 -1.14740245 -1.33545468 -0.87410912 -1.33545468 -0.59598343
  -1.26073114 -2.19722458 -2.08171169 -0.69088217 -0.95324644 -1.17638999
  -0.2368424  -1.61170806 -0.33647

## Part  D: Classify e-mails

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

In [10]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    
    print(new_data)
    
    class_predictions = np.array([])
    
    # Looping through each row in new_data
    for i, row in enumerate(new_data):
        # Summing the log_class_conditional_likelihoods for all the features in the dataset
        row_times_class_conditionals = row*log_class_conditional_likelihoods
        row_class_conditional_likelihoods_sum = np.sum(row_times_class_conditionals, axis=1)
        
        # Adding the row_class_conditional_likelihoods_sum to the log_class_priors
        log_total = row_class_conditional_likelihoods_sum + log_class_priors
        class_predictions = np.append(class_predictions, (log_total[0] < log_total[1]))
    
    return class_predictions

In [11]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [0. 0. 1. ... 1. 1. 1.]]


Now test your `predict` function by classifying messages. You can do this to the *training* data, but you should also try it on the *testing* data. 

In [64]:
# Check accuracy
true_classes = training_spam[:, 0]
training_set_accuracy = np.mean(np.equal(class_predictions, true_classes))
print(f"Accuracy on the training set: {training_set_accuracy}")

Accuracy on the training set: 0.814


Once you are done, you can move the code into the main assignment notebook.

One way to do this is to follow the rough structure of the class that already exists in that notebook. You can use the `train` method to pass in the data and perform all of the steps before the prediction. Note that you may need to adjust  You should store data in instance variables, e.g. `self.log_class_priors` and `self.log_class_conditional_likelihoods`. This means that then you can set up the `predict` method to match the one above without needing to pass in the additional variables. **Important:** the predict method must only take a single variable as a parameter (the one called `new_data`) in the skeleton code above.

In [ ]:
# This skeleton code simply classifies every input as ham
#
# Here you can see there is a parameter k that is unused, the
# point is to show you how you could set up your own. You can
# also see a train method that does nothing here
# but your classifier would probably do the main work here. 
# Modify this code as much as you like so long as the 
# accuracy test in the cell below runs

class MyClassifier:
    def __init__(self, k):
        # K shall be used to define the number of classes in the classification problem
        self.k = k
        # Declaring an array called log_class_priors which will hold the log of the class priors
        self.log_class_priors = np.array([])
        # Declaring an array called theta which will hold the log of the class conditional feature likelihoods
        self.theta = np.array([])
        
    def estimate_log_class_priors(self, data):
        """
        Given a data set with binary response variable (0s and 1s), 
        calculate the logarithm of the empirical class priors,
        that is, the logarithm of the proportions of 0s and 1s:
            log(p(C=0)) and log(p(C=1))

        :param data: a numpy array of length n_samples
                     that contains the binary response (coded as 0s and 1s).

        :return log_class_priors: a numpy array of length two
        """
        ### YOUR CODE HERE...

        # Defining np array
        out = np.array([])
        # Finding the probability of a ham email (c = 0)
        out = np.append(out, (len(data) - np.count_nonzero(data))/len(data))
        # Finding probability of a spam email (c = 1)
        out = np.append(out, np.count_nonzero(data)/len(data))
        # Getting the log of each element in out
        out = np.log(out)

        # Returning the np array
        return out
    
    def estimate_log_class_conditional_likelihoods(self, input_data, labels, alpha=1.0):
        """
        Given input_data of binary features (words) and labels 
        (binary response variable (0s and 1s)), calculate the logarithm 
        of the empirical class-conditional likelihoods, that is,
        log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

        Assume a multinomial feature distribution and use Laplace smoothing
        if alpha > 0.

        :param input_data: a two-dimensional numpy-array with shape = [n_samples, n_features]
                           contains binary features (words)
        :param labels: a numpy array of length n_samples 
                       contains response variable

        :return theta:
            a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
            logarithm of the probability of feature i appearing in a sample belonging 
            to class j.
        """
        ### YOUR CODE HERE...

        # Getting the magnitude of the dimensions
        n_samples = input_data.shape[0]
        n_features = input_data.shape[1]

        # Isolating all the lines in input_data where the class label == 0
        # np.squeeze removes all dimensions of size 1
        class_zeros = np.squeeze(input_data[np.where(labels == 0), :])
        
        # Isolating all the lines in input_data where the class label == 1
        class_ones = np.squeeze(input_data[np.where(labels == 1), :])

        # Calculating the empircal conditional liklihoods for class = 0 (c == 0)
        class_zeros_conditionals = ((np.count_nonzero(class_zeros, axis=0)) + alpha) / ((n_samples - np.count_nonzero(labels)) + n_features*alpha)
        log_class_zeros_conditionals = np.log(class_zeros_conditionals)

        # Calculating the empircal conditional liklihoods for class = 1 (c == 1)
        class_ones_conditionals = (np.count_nonzero(class_ones, axis=0) + alpha) / (np.count_nonzero(labels) + n_features*alpha)
        log_class_ones_conditionals = np.log(class_ones_conditionals)
        
        # Appending each of the class conditional likelihoods to an array called 'theta'
        theta = np.array([log_class_zeros_conditionals, log_class_ones_conditionals])

        return theta
        
    def train(self, train_data, train_labels):
        
        # Calculating the log of the probability of each class
        self.log_class_priors = self.estimate_log_class_priors(train_labels)
        # Calculating the log of the class conditional feature likelihoods
        # Remember to change alpha to optimise the training process
        self.theta = self.estimate_log_class_conditional_likelihoods(train_data, train_labels, alpha=100)
        
    def predict(self, test_data):
        """
        Given a new data set with binary features, predict the corresponding
        response for each instance (row) of the new_data set.

        :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
        :param log_class_priors: a numpy array of length 2.
        :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
            theta[j, i] corresponds to the logarithm of the probability of feature i appearing
            in a sample belonging to class j.
        :return class_predictions: a numpy array containing the class predictions for each row
            of new_data.
        """
        ### YOUR CODE HERE...

        class_predictions = np.array([])

        # Looping through each row in new_data
        for i, row in enumerate(test_data):
            # Summing the log_class_conditional_likelihoods for all the features in the dataset
            row_times_class_conditionals = row*self.theta
            row_class_conditional_likelihoods_sum = np.sum(row_times_class_conditionals, axis=1)

            # Adding the row_class_conditional_likelihoods_sum to the log_class_priors
            log_total = row_class_conditional_likelihoods_sum + self.log_class_priors
            class_predictions = np.append(class_predictions, (log_total[0] < log_total[1]))

        return class_predictions

# Change k to define the number of classes in the classification problem
spam_classifier = MyClassifier(k=2)
spam_classifier.train(train_data=training_spam[:, 1:], train_labels=training_spam[:, 0])